# DPP

行列式点过程多样性算法，Determinantal Point Process.

## 行列式

- 二阶行列式
  $$
  \left|\begin{array}{cccc} a_{00} & a_{01} \\a_{10} & a_{11} \\\end{array}\right| = a_{00}a_{11} - a_{01}a_{10}
  $$
  

- 三阶行列式
  $$
  \left|\begin{array}{cccc} a_{00} & a_{01} & a_{02}\\a_{10} & a_{11} & a_{12}\\a_{20} & a_{21} & a_{22}\\\end{array}\right| = a_{00}\left|\begin{array}{cccc} a_{11} & a_{12} \\a_{21} & a_{22} \\\end{array}\right| - a_{01}\left|\begin{array}{cccc} a_{10} & a_{12} \\a_{20} & a_{22} \\\end{array}\right| + a_{02}\left|\begin{array}{cccc} a_{10} & a_{11} \\a_{20} & a_{21} \\\end{array}\right|
  $$

- 通项
  $$
  \left|\begin{array}{cccc} a_{00} & a_{01} & \cdots & a_{0n}\\a_{10} & a_{11} & \cdots & a_{1n}\\\vdots & \vdots & \ddots & \vdots\\a_{n0} & a_{n1} & \cdots & a_{nn}\\\end{array}\right| = \sum_{j_{1},j_2,\cdots j_{n}}(-1)^ta_{1j_1}a_{2j_2}\cdots a_{nj_n}
  $$
  其中$j_{1},j_2,\cdots j_{n}$表示$1\to n$ 的一个全排列，$t$为这个全排列的逆序数。

- 记法：$det(A)$表示矩阵$A$的行列式

- 直观理解

  将每行看成一个向量，行列式的大小反映了向量的夹角大小。例如:  
    $$
    \left|\begin{matrix}
    1 & 0 \\
    1 & 0 \\
    \end{matrix}\right| = 0
    $$
    $$
    \left|\begin{matrix}
    1 & 0 \\
    0 & 1 \\
    \end{matrix}\right| = 1
    $$
  两个向量的夹角分别为$0^{\circ}$和$90^{\circ}$

- Cholesky Decomposition (Cholesky分解)

  半正定矩阵$A=A^\top$且$x^\top Ax>0$可分解成$A = L \cdot L ^ \top$，其中$L$是一个下三角矩阵

  在`numpy`中为`L = np.linalg.cholesky(A)`

## DPP的构造

对于离散集合$Z = {I_1, I_2, \cdots},I_M$，对于$Z$的每一个子集$Y$，构造一个矩阵$L_Y$，其中$L_{ij} = <V_i, V_j>$，其中$V_i$表示带相关性权重的item向量: $V_i = r_i \cdot v_i$，其中$r_i$为item和query/user的相关性（如CTR)，$v_i$为item的向量。

假设采用余弦相似度那么，则有
$$
det(L) = det(\left| <V_i, V_j> \right|) = det(\left| r_ir_j<v_i, v_j>\right|) = det(\left| r_ir_js_{ij}\right|)
$$
根据
$$
L_{1j_1}L_{2j_2}\cdots L_{nj_n} = \prod_{i \in M}s_i \prod_{i \in M}r_i^2
$$
和的公式，可以把$r_ir_j$提取出来
$$
det(L_Y) = \prod_{i \in M}r_i^2 \cdot det(S_Y)
$$
$det(L_Y)$就变成了相关性$\prod_{i \in M}r_i^2$和多样性$det(S_Y)$的乘积了，至于为什么$det(S_Y)$反映了多样性，我们可以直接看计算：

In [1]:
from typing import Dict
import numpy as np

v1 = np.array([1, 2, 3])
v2 = np.array([1, 2, 4])
v3 = np.array([1, 3, 3])

s12 = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
s13 = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v3))
s23 = np.dot(v1, v2) / (np.linalg.norm(v2) * np.linalg.norm(v3))

S1 = np.array([[1, s12, s13], [s12, 1, s23], [s13, s23, 1]])
np.linalg.det(S1)

-0.03472968134622267

In [2]:
v1 = np.array([1, 2, 6])
v2 = np.array([9, 2, 4])
v3 = np.array([1, 7, 3])

s12 = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
s13 = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v3))
s23 = np.dot(v1, v2) / (np.linalg.norm(v2) * np.linalg.norm(v3))

S1 = np.array([[1, s12, s13], [s12, 1, s23], [s13, s23, 1]])
np.linalg.det(S1)

0.2883770807837294

也可以看推导，在有两个向量的情况下:
$$
y = det(S) = s_{11}s_{22}-s_{12}s_{21} = 1 - s_{12}^2
$$
假设有三个向量，
$$
y = det(S) = 1 + s_{12}s_{23}s_{13}+s_{13}s_{12}s_{23} - s_{23}s_{23} - s_{12}s_{12} - s_{13}s_{13} = 1 + 2abc -a^2-b^2-c^2
$$
（不会推了，上数值计算）

In [3]:
a = np.linspace(-1, 1, 100)
b = np.linspace(-1, 1, 100)
c = np.linspace(-1, 1, 100)

y = 1 + 2 * a * b * c - np.square(a) - np.square(b) - np.square(c)
print(np.argmax(y))

50


## 贪婪算法求近似解

优化目标
$$
log det(L_{Y_g})= \sum_{i \in g} log (r_i^2) + log det(S_{Y_g})
$$
求解过程：每次从候选商品中选取一个item$j$满足
$$
j = Arg \ \underset{D_i \in R \backslash Y_g}{max} logdet(Y_{g\cup \{i\}}) - logdet(Y_g)
$$
即每次取的item要让前后的$logdet$尽可能相差大。

对$Y_g$做Cholesky分解：
$$
Y_g = VV^\top
$$


对$Y_{g\cup \{i\}}$做Cholesky分解：
$$
Y_{g\cup \{i\}} = \left[ \begin{matrix} L_{Y_g} & L_{Y_g,i} \\ L_{i,Y_g} & L_{ii}\end{matrix} \right] = \left[ \begin{matrix} V & 0 \\ c_i & d_i\end{matrix} \right]\left[ \begin{matrix} V^\top & c_i^\top \\ 0 & d_i\end{matrix} \right]
$$
根据矩阵乘法：
$$
L_{Y_g, i} = Vc_i^\top\\L_{ii} = c_ic_i^\top + d_i^2 \rightarrow d_i^2 = L_{ii} - \|c_i\|_2^2
$$
由行列式的性质$det(AB) = det(A)det(B)$和$Y_{g\cup \{i\}}$的Cholesky分解可得：
$$
det(Y_{g\cup \{i\}}) = det(V)d_idet(V^\top d_i) = det(VV^\top)d_i^2 = det(Y_g)d_i^2
$$
因此优化函数：
$$
logdet(Y_{g\cup \{i\}}) - logdet(Y_g) = log(det(Y_g)d_i^2) - logdet(Y_g) = log(d_i^2) = log(L_{ii} - \|c_i\|_2^2)
$$
当优化函数$log(\|c_i\|_2^2 - L_{ii})$求出了解$j$，则$c_j$和$d_j$也可以求出来，以更新$L_{g \cup \{i\}}$:
$$
L_{g \cup \{i\}} = \left[ \begin{matrix} V & 0 \\ c_j & d_j\end{matrix} \right]\left[ \begin{matrix} V^\top & c_j^\top \\ 0 & d_j\end{matrix} \right]
$$
在求解新的$c_i$和$d_i$时，分别记为$c'_i$, $d'_i$，则有：
$$
L_{Y_g,i} = Vc_i^\top
$$
$$
L_{Y_g\cup\{j\},i} = V'c{'}_{i}^\top =  \left[ \begin{matrix} V & 0 \\ c_j & d_j\end{matrix} \right]c{'}_{i}^\top = \left[\begin{matrix} L_{Y_g,i \\ L_{ji}} \end{matrix}\right] \\
$$
$$
c{'}_i = \left[\begin{matrix}c_i & e_i\end{matrix}\right]
$$
$$
e_i= L_{ji}-\left<c_i,c_j\right>/d_j
$$
$$
d{'}_i = d_i^2 - e_i^2
$$

In [21]:
class DetPointProcess:
    def __init__(self, item_sim_mat: np.ndarray):
        self.item_sim_mat = item_sim_mat
        self.epsilon = 0.01
    def calculate(self, item_scores: Dict[int, float], top_k=10):
        """

        :param item_scores: {item_index: score}
        :param top_k:
        :return:
        """
        z = list(item_scores.keys())
        kernel_mat = np.zeros((len(z), len(z)))
        for i in range(len(z)):
            for j in range(len(z)):
                kernel_mat[i, j] = item_scores[z[i]] * self.item_sim_mat[z[i], z[j]]
        # print(kernel_mat)
        # re_index_map = {i: z[i] for i in range(len(z))}
        c = np.zeros((top_k, len(z)))
        d = np.copy(np.diag(kernel_mat))
        j = np.argmax(d)
        y_g = [j]
        re_z = list(range(len(z)))
        it = 0
        while len(y_g) < top_k:
            rest = set(re_z).difference(set(y_g))
            for i in rest:
                if it == 0:
                    ei = kernel_mat[j, i] / np.sqrt(d[j])
                else:
                    ei = kernel_mat[j, i] / np.dot(c[:it, j], c[:it, i])
                c[it, i] = ei
                d[i] = d[i] - ei * ei
            d[j] = -np.inf
            j = np.argmax(d)
            # if d[j] < self.epsilon:
                # break
            y_g.append(j)
            it += 1
        return [z[i] for i in y_g]

In [22]:
# 测试
item_sim_mat = np.array(
    [[1., 0.9, 0.6, 0.3], 
     [0.9, 1., 0.3, 0.7], 
     [0.6, 0.3, 1., 0.8], 
     [0.3, 0.7, 0.8, 1.]])
item_scores = {0: 0.6, 1: 0.5, 2: 0.8, 3: 0.9}

In [23]:
dpp_obj = DetPointProcess(item_sim_mat)
# re-rank
export_yg = dpp_obj.calculate(item_scores, 4)
print(export_yg)

[0.519 0.059 0.224  -inf]
[       -inf -8.10426531 -2.55377778        -inf]
[      -inf -8.1063425       -inf       -inf]
[3, 0, 2, 1]


- 这里借用了MMR的数据，可以看出来在这个sample上重排的结果和MMR相同，同样也可能存在局部最优的情况，但由于优化的是
$det(L_Y) = \prod_{i \in M}r_i^2 \cdot det(S_Y)$这个乘积，因此不方便直接修改优化目标


